# **Lab 1: Matrix Algorithms**
**Pablo Aravena**

# **Abstract**

 We are faced with the task of programming Python implementations for common linear algebra operations. For the code, the author took an approach of transparency line-by-line (with comments for almost each of them) while still trying to condense most functionality in Python one-liners.
 
 Afterwards, the functions are tested against single cases (for easier visualization) and finally with a bulk of randomized test cases (dimension-wise and element-wise), in which we only show the rate of successful cases against the total of them. Verifying those test cases was easy as numpy already implements those very same functions.

# **About the code**

In [3]:
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2019 Pablo Aravena (pjan2@kth.se)

# Based on the template by Johan Hoffman (jhoffman@kth.se)
# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

In [0]:
# Load neccessary modules.
from google.colab import files

import numpy as np
from math import sqrt

# **Introduction**

We implement 5 different functions for basic linear algebra concepts, the first one being the dot product (or scalar product) between vectors, a matrix - vector product, matrix - matrix product and the extras of euclidean norm and distance for vectors. The author included extra code for special cases or problems related with the space the objects reside in that could appear later in the randomized rows/cols test cases.

# **Methods**

#### Vector - Vector  Scalar Product


$ d  = x \cdot y = \sum_{1 \leq i \leq n} x_i \cdot y_i $, where $x, y \in \mathbb{R}^n$ and $d \in \mathbb{R}$

All numpy vectors (arrays) should have $1$ row and $n$ columns.

In [0]:
# dot product
def scalarProduct(x, y):
    # check if both vectors aren't on different vector spaces
    valid = True if x.shape == y.shape else False
    
    # if correct number of rows and cols
    if valid:
        sum = 0
        # multiply all col values
        for i in range(x.shape[1]):
            sum += x[0][i] * y[0][i]
        
        # return scalar product
        return sum
        
    # invalid rows and cols, no output
    return


#### Matrix - Vector Product

We reused code from the previous implementation, as to not write similar pieces twice and build up from simple constructs. The main idea is to multiply every row with the same $x$ vector, using the scalar product with every row vector from the matrix.

$ A \cdot x = $ $ 
   \begin{bmatrix} 
        A_1 \\
        A_2 \\
        \vdots \\
        A_m
   \end{bmatrix}
$
$
   \cdot x = \begin{bmatrix}
      A_1 \cdot x \\
      A_2 \cdot x \\
      \vdots \\
      A_m \cdot x \\
   \end{bmatrix}
$, where $ x \in \mathbb{R}^{n x 1}, A \in \mathbb{R}^{m x n}$ and $ A_i \in \mathbb{R}^{1 x n}$, $ i \in \{1,2,\dots,m\}$

In [0]:
# matrix vector product
def matrixVectorProduct(x, A):
    # check validity of rows and columns for vector and matrix multiplication
    valid = True if A.shape[1] == x.shape[0] else False
    
    # b vector ends up in R^nx1 (n x k * k x 1 = n x 1)
    b = np.zeros((A.shape[0], x.shape[1]))
    
    # multiply only if they posess correct rows and columns
    if valid:
        # current column
        col = 0
        
        # go through matrix rows
        for row in range(A.shape[0]):
            # re use scalar product function
            b[col, x.shape[1] - 1] = scalarProduct(np.array([x[:, 0]]), np.array([A[row]]))
            col += 1
            
        # return b vector if scalar is not None
        if None not in b:
            return b
    
    # empty result
    return


#### Matrix - Matrix Product

The author reused again the previous function as to take advantage of it, multiplying every row of the first matrix ($A$) with the columns of the second one ($B$) and then replacing them as columns on the new generated matrix.

\begin{equation*}
A \cdot B = 
\begin{bmatrix}
    A_1 \\
    A_2 \\
    \vdots \\
    A_m
\end{bmatrix}
\begin{bmatrix}
    B_1 &
    B_2 &
    \dots &
    B_j
\end{bmatrix}=
\begin{bmatrix}
    \begin{pmatrix}
       A_1 \cdot B_1 \\
       A_2 \cdot B_1 \\
       \vdots \\
       A_m \cdot B_1
    \end{pmatrix} &
    \begin{pmatrix}
       A_1 \cdot B_2 \\
       A_2 \cdot B_2 \\
       \vdots \\
       A_m \cdot B_2
    \end{pmatrix} &
    \dots &
    \begin{pmatrix}
       A_1 \cdot B_j \\
       A_2 \cdot B_j \\
       \vdots \\
       A_m \cdot B_j
    \end{pmatrix}
\end{bmatrix}
\end{equation*}

where $A \in \mathbb{R}^{mxn}$, $B \in \mathbb{R}^{nxj}$, $A_i \in \mathbb{R}^{1xn}$ and $B_k \in \mathbb{R}^{nx1}$, with
$i \in \{1,\dots, m\}$ and $k \in \{1,\dots, j\}$

In [0]:
# matrix to matrix product
def matrixMatrixProduct(A, B):
    # check validity of rows and cols for both matrices
    valid = True if A.shape[1] == B.shape[0] else False
    
    # multiply only if valid rows and cols
    if valid:
        # matrix with results
        new_matrix = np.zeros((A.shape[0], B.shape[1]))
        
        # run through first matrix rows
        for col in range(new_matrix.shape[1]):
            # get column vector as row vector and shift it to column
            temp_B = B[:, col].reshape((B.shape[0], 1))
            
            # reuse Matrix-Vector product function
            new_col = matrixVectorProduct(temp_B, A)
            
            # check first if there wasn't an error on any calculation and replace column with vector
            if None not in new_col:
                new_matrix[:, col] = new_col[:, 0]
            # otherwise immediately return None
            else:
                return
            
        # return product matrix
        return new_matrix
    
    # return None as failure
    return
    

#### Euclidean Norm

We implemented the next operation:

$||x|| = \sqrt{x_1^2 + x_2^2 + \dots + x_n^2}$, where $x \in \mathbb{R}^n$ and $||x|| \in \mathbb{R}$

It expects a numpy array of $1$ row and $n$ columns.

In [0]:
# euclidean norm
def euclideanNorm(x):
    sum = 0
    
    # go through every element in vector
    for col in range(x.shape[1]):
        # sum = x0^2 + x1^2 + ... + xn^2
        sum += x[0, col]**2
        
    # return square root
    return sqrt(sum)
    

#### Euclidean Distance

The author reused the same euclidean norm function used before to calculate the distance, as:

\begin{align*}
|x - y| = |(x_1, x_2, \dots, x_n) - (y_1, y_2, \dots, y_n)| =\\
|(x_1-y_1, x_2-y_2,\dots, x_n-y_n)| = \\
\sqrt{(x_1 - y_1)^2 + (x_2 - y_2)^2 + \dots + (x_n - y_n)^2}
\end{align*}

Here both $x$ and $y$ should be numpy arrays of $1$ row and $n$ columns.

In [0]:
# euclidean distance
def euclideanDistance(x, y):
    valid = True if x.shape[1] == y.shape[1] else False
    
    # if both vectors share the same vector space
    if valid:
        # return euclidean norm of vector substraction 
        return euclideanNorm(x - y)
            
    # return None if they mismatch number of cols
    return


#### Traversal and Main Tester Functions

The author made a traversal function for general testing, in which we only need the function id and whether the parameters to be used are vectors or not. After that, a main testing function leverages the previous one inside some loops to create 1000 test cases for every function with randomized dimensions and elements on the vectors/matrices, and verifies the results with correct ones provided by numpy implementations of the same ones.
At the end we present the rate of successful test cases against the known correct ones.

In [0]:
# generic tester function (helper for main tester)
def testHelper(func_id, is_p1_vector, is_p2_vector):
    # random generated rows and columns accordingly to matrix/vector (m x n -- k x j)
    # vector -- vector
    #    1 x n -- 1 x j
    # matrix -- vector
    #    m x n -- n x 1
    # matrix -- matrix
    #    m x n -- n x j
    m = 1 if is_p1_vector else np.random.randint(1, 10)
    n = np.random.randint(1, 10)
    k = 1 if (is_p1_vector and is_p2_vector) else n
    j = 1 if is_p2_vector else np.random.randint(1, 10)

    
    # generate vector or matrix for test cases
    x = np.random.rand(m, n)
    y = np.random.rand(k, j)
    
    # vector -- vector (scalar) product 
    if func_id == 0:
        test_val = scalarProduct(x, y)
        try:
            real_val = np.inner(x, y)[0][0]
        except:
            real_val = None
        
    # matrix -- vector product
    elif func_id == 1:
        test_val = matrixVectorProduct(y, x)
        real_val = x @ y
        
    # matrix -- matrix product
    elif func_id == 2:
        test_val = matrixMatrixProduct(x, y)
        real_val = x @ y
        
    # vector euclidean norm
    elif func_id == 3:
        test_val = euclideanNorm(x)
        real_val = np.linalg.norm(x)
        
    # vector -- vector euclidean distance
    else:
        test_val = euclideanDistance(x, y)
        real_val = np.linalg.norm(x - y) if x.shape[1] == y.shape[1] else None
    
    # if results are scalars
    if func_id in [0, 3, 4]:
        return test_val == real_val
    
    # otherwise compare matrices (with default relative and absolute tolerance)
    return np.allclose(test_val, real_val)



# main tester function
def testCases():
    # truth values dictionary for different functions
    # where 1 implies positional parameter is a vector
    # and 0 implies is a matrix
    mat_or_vec = {
                '0': [1, 1],
                '1': [0, 1],
                '2': [0, 0],
                '3': [1, 1],
                '4': [1, 1]
                }

    # dictionary with id's mapped to function names
    functions = {'0': 'scalarProduct',
                 '1': 'matrixVectorProduct',
                 '2': 'matrixMatrixProduct',
                 '3': 'euclideanNorm',
                 '4': 'euclideanDistance'
                }
    
    # total of test cases for every function
    total = 1000

    # create test cases for each function
    for f_id in range(5):
        success = 0
        # run total of test cases for every one of them
        for test_case in range(total):
            # call traveler test helper function
            res = testHelper(f_id, mat_or_vec[str(f_id)][0], mat_or_vec[str(f_id)][1])
        
            # if test case and real value are the same
            if res:
                # add up to the success counter
                success += 1
        
        # rate of successful test cases
        success_rate = 100.0 * success / total
        print(f"[+] Success rate for \'{functions[str(f_id)]}\': {success_rate}%")
        

# **Results**

The next segment of code shows very simple test cases for each function and then bulk tests for each one. It works as intended for every one of them.

In [9]:
print("\t Singular Test Cases")

# basic vector -- vector (scalar) product test case
x, y = np.array([[1,1,1,1]]), np.array([[1,1,1,1]])
print("-"*50)
print(f"[+] Scalar product: {scalarProduct(x, y)}")
print(f"[-] Real scalar product: {np.inner(x ,y)[0][0]}")
print("-"*50)


# basic matrix -- vector product test case
x = np.array([[1], [2], [3], [4]])
A = np.ones((4,4))

print("-"*50)
print(f"[+] Matrix-Vector Product: \n{matrixVectorProduct(x, A)}")
print(f"[-] Real Matrix-Vector product is: \n{A @ x}")
print("-"*50)


# basic matrix -- matrix product test case
A = np.array([[1,1,1], [1,1,1], [1,1,1]])
B = np.array([[1,1,1,1,2], [1,1,1,1,2], [1,1,1,1,2]])

print("-"*50)
print(f"[+] Matrix to Matrix product: \n{matrixMatrixProduct(A, B)}")
print(f"[-] Real Matrix-Matrix product is: \n{A @ B}")
print("-"*50)


# basic euclidean norm test case
x = np.array([[1,4,5,8]])
print("-"*50)
print(f"[+] Norm is: {euclideanNorm(x)}")
print(f"[-] Real euclidean norm is: {np.linalg.norm(x)}")
print("-"*50)


# basic euclidean distance test case
x = np.array([[1, 2, 3 ,4]])
y = np.array([[1, 1, 1, 1]])

print("-"*50)
print(f"[+] Euclidean distance between x and y is: {euclideanDistance(x,y)}")
print(f"[-] Real euclidean distance is: {np.linalg.norm(x - y)}")
print("-"*50)

# make generic tests
print("\t Bulk Test Cases' Stats (1000 per Function)\n")
testCases()
print("-"*50)

	 Singular Test Cases
--------------------------------------------------
[+] Scalar product: 4
[-] Real scalar product: 4
--------------------------------------------------
--------------------------------------------------
[+] Matrix-Vector Product: 
[[10.]
 [10.]
 [10.]
 [10.]]
[-] Real Matrix-Vector product is: 
[[10.]
 [10.]
 [10.]
 [10.]]
--------------------------------------------------
--------------------------------------------------
[+] Matrix to Matrix product: 
[[3. 3. 3. 3. 6.]
 [3. 3. 3. 3. 6.]
 [3. 3. 3. 3. 6.]]
[-] Real Matrix-Matrix product is: 
[[3 3 3 3 6]
 [3 3 3 3 6]
 [3 3 3 3 6]]
--------------------------------------------------
--------------------------------------------------
[+] Norm is: 10.295630140987
[-] Real euclidean norm is: 10.295630140987
--------------------------------------------------
--------------------------------------------------
[+] Euclidean distance between x and y is: 3.7416573867739413
[-] Real euclidean distance is: 3.7416573867739413


# **Discussion**

The success rate for every function shows that they work as intended. A collaboration with Fabián Levicán and Felipe Vicencio was made for sharing some interesting ideas in the making of the tests and other subjects related to the reuse of code.